<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>


Getting required libraries

In [8]:
!pip install dash

Defaulting to user installation because normal site-packages is not writeable


In [9]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

C:\Users\Joshua\AppData\Local\Temp\ipykernel_15724\1855361594.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\Joshua\AppData\Local\Temp\ipykernel_15724\1855361594.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [10]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
spacex_df = pd.read_csv(url)

spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [11]:
sites_df = spacex_df.groupby("Launch Site")["Launch Site"].count().reset_index(name='counts')

sites_df.head()

,Launch Site,counts
0,CCAFS LC-40,26
1,CCAFS SLC-40,7
2,KSC LC-39A,13
3,VAFB SLC-4E,10


In [91]:
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

dropdown_options = [{'label': 'All Sites', 'value': 'ALL'}]

for index, row in sites_df.iterrows():
    dropdown_options.append({'label': row["Launch Site"], 'value': row["Launch Site"]})

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(
    children=[
        html.H1(
            'SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36','font-size': 40}
        ),
        # TASK 1: Add a dropdown list to enable Launch Site selection
        html.Div( 
            dcc.Dropdown(id='site-dropdown',
                options=dropdown_options,
                value='ALL',
                placeholder="Select Launch Site",
                searchable=True
            )
        ),
        html.Br(),

        # TASK 2: Add a pie chart to show the total successful launches count for all sites
        # If a specific launch site was selected, show the Success vs. Failed counts for the site
        html.Div(dcc.Graph(id='success-pie-chart')),
        html.Br(),

        html.P("Payload range (Kg):"),
        # TASK 3: Add a slider to select payload range
        html.Div(
            dcc.RangeSlider(id='payload-slider',
                min=0, max=10000, step=500,
                marks={
                    0: '0',
                    2500: '2500',
                    5000: '5000',
                    7500: '7500',
                    10000: '10000',
                },
                value=[min_payload, max_payload])
        ),

        # TASK 4: Add a scatter chart to show the correlation between payload and launch success
        html.Div(dcc.Graph(id='success-payload-scatter-chart')),
    ]
)

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback( 
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(
            spacex_df, 
            values='class', 
            names='Launch Site', 
            title='Successful Launchs By Site'
        )
        return fig
    else:
        filtered_df = spacex_df[spacex_df["Launch Site"] == entered_site ]
        fig = px.pie(
            filtered_df.groupby("class")["class"].count().reset_index( name = 'counts'), 
            values='counts', 
            names='class', 
            title='Launch Outcome Share'
        )
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback( 
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'), 
     Input(component_id="payload-slider", component_property="value")]
)
def get_scatter_chart(entered_site, payload_range):
    low, high = payload_range
    if entered_site == 'ALL':
        filtered_df = spacex_df[ 
            (spacex_df["Payload Mass (kg)"] >= low)
            & (spacex_df["Payload Mass (kg)"] <= high)
        ]
        fig = px.scatter(
            filtered_df, 
            x="Payload Mass (kg)", 
            y="class", 
            color="Booster Version Category",
            title="Correlation Between Payload and Success For All Sites"
        )
        return fig
    else:
        filtered_df = spacex_df[ 
            (spacex_df["Launch Site"] == entered_site)
            & (spacex_df["Payload Mass (kg)"] >= low)
            & (spacex_df["Payload Mass (kg)"] <= high)
        ]

        fig = px.scatter(
            filtered_df, 
            x="Payload Mass (kg)", 
            y="class", 
            color="Booster Version Category",
            title="Correlation Between Payload and Success For: " + entered_site
        )
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()